Import the libraries needed

In [1]:
import pandas as pd
#from pandas.compat import StringIO
from pandas.tseries.offsets import DateOffset

from datetime import datetime
import psutil

import numpy as np
import os
import pyodbc
pyodbc.drivers()

import csv

from sklearn.preprocessing import StandardScaler
import seaborn as sns

pd.options.display.max_rows = 1000

# For the Yahoo Finance api
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override() 


C:\Users\m033\Anaconda3\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (3.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn(


### Database connections and SQL to retrieve the various data needed.

In [2]:
SQL_server = 'xxx\yyy'
Reporting_database = 'zzz'

pd.set_option('max_row', None)
#pd.options.display.max_rows = 1000

# Set a value for the home folder.
home_folder = "."

# Set values for the various paths.
input_path = home_folder + "\data"
external_data = home_folder + "\external_data"


print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')
os.path.abspath(os.getcwd())

2024-08-05 21:59:15.340506
svmem(total=16756752384, available=9159553024, percent=45.3, used=7597199360, free=9159553024)


'C:\\Users\\m033\\OneDrive - GWLE\\Documents\\GitHub\\Data-Science-Thesis'

# 1. Get supplementary client features.

  -  Identified in Lit Review.# 1. Read back the population with switches data.

In [3]:
# READ IN directly from DW
def customer_data(server, database):
    conn = pyodbc.connect(
         'DRIVER={ODBC Driver 17 for SQL Server};'
         'SERVER='+server+';'
         'DATABASE='+database+';'
         'Trusted_Connection=yes;')
  
    query = """SELECT pol.Policy_Number
      ,pol.Risk_Commencement_Date AS RiskCommDt
--      ,pol.Cover_End_Date AS CoverEndDt
      ,pa.VIP_Type_Id          AS VIPType
      ,pa.Nationality_Code     AS Nationality
      ,CASE WHEN pa.Is_Smoker = 'True' THEN 1
            ELSE 0 
       END as IsSmoker
      ,pa.Politically_Exposed_Id AS PoliticallyExposed
      ,pa.Birth_Place               As BirthPlace
      ,a.Postcode                   AS AddressPostcode
	  ,pa.Sex_Id                    AS Gender
	  ,pa.Marital_Status_Id         AS MaritalStatus
	  ,pla.Pension_Disability_Level AS PensionDisability
	  ,pla.WOP_Disability_Level     AS WOPDisability
      ,a.City                       AS AddressCity
	  ,pa.Date_Of_Birth             AS DOB
--      ,DATEDIFF(month, GETDATE(),pol.Cover_End_Date) AS TermRemainingMths
--      ,DATEDIFF(month, GETDATE(),pol.Cover_End_Date) AS TermRemainingMths
      ,pa.Partner_Type_De           AS PartnerType
      ,pa.Is_EU_Sanctioned          AS EUSanctioned
      ,a.District                   As AddressDistrict
FROM CLT_Policy pol,
		 CLT_Product prd,
		 CLT_Policy_Life_Assured pla,
	  	 CLT_Partner pa,
		 CLT_Address a
WHERE prd.Product_Id = pol.Product_Id
AND   pla.Policy_Id = pol.Policy_Id
AND   pol.Policy_id = pla.Policy_Id
AND   pla.Partner_Id = pa.Partner_Id
AND   pa.Partner_Number = a.Partner_Number
AND   prd.Product_Code IN ('FKP_2006V1','FKP_2006V2','FKP_2008V1','FKP_2010V1','FKP_2012V1','FKP_2017V1',
                           'GNPP_2016V1','GNPP_2017V1','GNPP_2022V1','GNPP_2023V1',
                           'GNRP_2008V1','GNRP_2010V1','GNRP_2012V1','GNRP_2017V1','GNRP_2022V1',
                           'XRPP_2005V1','XRPP_2008V1','XRPP_2010V1','XRPP_2012V1','XRPP_2015V1');"""    
    
    
    return pd.read_sql(query, conn)

df_customer = customer_data(SQL_server, Reporting_database)

# Types and row count
# print stats.
print(len(df_customer.index))
now = datetime.now()
print(now)
print('==========================')

print(df_customer.dtypes)

# No displays to protect PII data
#df_customer.head(5)

Error: ('HY000', '[HY000] [Microsoft][ODBC Driver 17 for SQL Server]SQL Server Network Interfaces: The system cannot contact a domain controller to service the authentication request. Please try again later.\r\n (-2146892976) (SQLDriverConnect); [HY000] [Microsoft][ODBC Driver 17 for SQL Server]Cannot generate SSPI context (-2146892976); [HY000] [Microsoft][ODBC Driver 17 for SQL Server]SQL Server Network Interfaces: The system cannot contact a domain controller to service the authentication request. Please try again later.\r\n (-2146892976); [HY000] [Microsoft][ODBC Driver 17 for SQL Server]Cannot generate SSPI context (-2146892976)')

## Drop obsolete or repeated features.

## Check for nulls

In [ ]:
df_customer.dtypes

In [ ]:
df_customer.isna().any()

## Replace any nulls before checking the distributions of values for each column.

In [4]:
df_customer['PensionDisability'] = df_customer['PensionDisability'].fillna(0).apply(np.int64)
df_customer['WOPDisability'] = df_customer['WOPDisability'].fillna(0).apply(np.int64)


NameError: name 'df_customer' is not defined

## Set the Postcode to int to facilitate the join.

In [5]:
df_customer['AddressPostcode'] = (
    pd.to_numeric(df_customer['AddressPostcode'],
                  errors='coerce')
      .fillna(0)
    )
df_customer['AddressPostcode'] = df_customer['AddressPostcode'].astype(int)
df_customer.dtypes

NameError: name 'df_customer' is not defined

# Read in the postcode data.

In [6]:
# giving directory name
filename = external_data + '\Postcode_State_Metadata.xlsx'

#df_Postcode = pd.read_excel(filename,sheet_name='Tabelle1')
df_Postcode = pd.read_csv(filename)
df_Postcode['Postal_Code'] = df_Postcode['Postal_Code'].astype(int)

print(df_Postcode.dtypes)

# No displays to protect PII data
#df_Postcode.head(2)

Postal_Code          int32
Postal_Code_Name    object
District            object
State               object
dtype: object


# Merge the address and the external postcode data.

In [7]:
# Left Join the list of campaign contract numbers to the main dataset.
df_customer = pd.merge(df_customer, df_Postcode, left_on = ['AddressPostcode'], right_on = ['Postal_Code'] , how = 'left')

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

# No displays to protect PII data
#df_customer.head(5)

NameError: name 'df_customer' is not defined

# Merge in the sinus milieus data.

In [8]:
# giving directory name
filename = external_data + '\sinus_milieu.csv'


df_Sinus_Milieus = pd.read_csv(filename)

print(df_Sinus_Milieus.dtypes)

# No displays to protect PII data
#df_Sinus_Milieus.head(2)

CONTRACT_NUMBER                 object
DATA_VERSION                   float64
POLICY_NUMBER                   object
VN_MILIEU_CODE                  object
VP_MILIEU_CODE                  object
VN_TARGET_GROUP_CODE            object
VP_TARGET_GROUP_CODE            object
VN_MILIEU_DESCRIPTION           object
VN_TARGET_GROUP_DESCRIPTION     object
VP_MILIEU_DESCRIPTION           object
VP_TARGET_GROUP_DESCRIPTION     object
dtype: object


# Merge in the Sinus data.

In [9]:
df_customer = pd.merge(df_customer, df_Sinus_Milieus, left_on = ['Policy_Number'], right_on = ['CONTRACT_NUMBER'] , how = 'left') 

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

print(df_customer.dtypes)


# No displays to protect PII data
#df_customer.head(5)

NameError: name 'df_customer' is not defined

## Drop all non predictive features and write to Excel

In [10]:
df_customer = df_customer.drop(['AddressPostcode','CONTRACT_NUMBER'],axis=1)

# Write out to a comma separated values file.

# giving directory name
filename = input_path + '\Client_data.csv'
df_customer.to_csv(filename, encoding='utf-8', index=False)        

# No displays to protect PII data
#df_customer.head(10)

NameError: name 'df_customer' is not defined

In [11]:
%who_ls DataFrame 

['df_Postcode', 'df_Sinus_Milieus']

In [12]:
del df_Postcode
del df_Sinus_Milieus
del df_customer



NameError: name 'df_customer' is not defined

In [13]:
print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

2024-08-05 22:01:54.141409
svmem(total=16756752384, available=9268568064, percent=44.7, used=7488184320, free=9268568064)
